In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: Averaging history and use FFN

Here, we average the full history of a path and concatenate it to the current embedding (the total number of features that are passed into the FFN is 2 * sbert_embeddings.shape[0]).

Here, we will run the hyperparameter search to implement the FFN with the same parameters as the standard FFN baseline on the sentence embeddings.

In [9]:
num_epochs = 100
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-4] #[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [10]:
ffn_mean_history, best_ffn_mean_history, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=None,
    data_split_seed=123,
    split_ids= torch.tensor(df_rumours['timeline_id'].astype(int)),
    k_fold=False,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


  0%|          | 0/1 [00:00<?, ?it/s]


[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups



100%|██████████| 1/1 [00:20<00:00, 20.33s/it]


[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the mean history for each item in the dataframe


100%|██████████| 5568/5568 [00:09<00:00, 560.70it/s]


[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups
[INFO] Splitting data by provided groups


In [11]:
ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id,input_dim
0,focal,0.6048,0.558703,"[0.7013301088270859, 0.4160756501182033]",0.206805,0.717675,0.710016,"[0.7571428571428572, 0.6628895184135978]","[32, 32]",0.2,0.0001,0,2,False,0.0,768
0,focal,0.6040,0.560070,"[0.6990881458966566, 0.42105263157894735]",0.206156,0.722420,0.715974,"[0.7587628865979382, 0.6731843575418994]","[32, 32]",0.2,0.0001,1,2,False,0.0,768
0,focal,0.5968,0.546581,"[0.6974789915966387, 0.39568345323741005]",0.209060,0.717675,0.710468,"[0.7561475409836067, 0.6647887323943663]","[32, 32]",0.2,0.0001,12,2,False,0.0,768
0,focal,0.5968,0.547659,"[0.6967509025270757, 0.39856801909307876]",0.212790,0.727165,0.719764,"[0.7653061224489794, 0.6742209631728046]","[32, 32]",0.2,0.0001,123,2,False,0.0,768
0,focal,0.5808,0.542424,"[0.674937965260546, 0.4099099099099099]",0.208568,0.722420,0.716581,"[0.7572614107883817, 0.6759002770083102]","[32, 32]",0.2,0.0001,1234,2,False,0.0,768


In [12]:
best_ffn_mean_history

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,input_dim
0,focal,0.6048,0.558703,"[0.7013301088270859, 0.4160756501182033]",0.206805,0.717675,0.710016,"[0.7571428571428572, 0.6628895184135978]","[32, 32]",0.2,0.0001,0,2,False,768
0,focal,0.6040,0.560070,"[0.6990881458966566, 0.42105263157894735]",0.206156,0.722420,0.715974,"[0.7587628865979382, 0.6731843575418994]","[32, 32]",0.2,0.0001,1,2,False,768
0,focal,0.5968,0.546581,"[0.6974789915966387, 0.39568345323741005]",0.209060,0.717675,0.710468,"[0.7561475409836067, 0.6647887323943663]","[32, 32]",0.2,0.0001,12,2,False,768
0,focal,0.5968,0.547659,"[0.6967509025270757, 0.39856801909307876]",0.212790,0.727165,0.719764,"[0.7653061224489794, 0.6742209631728046]","[32, 32]",0.2,0.0001,123,2,False,768
0,focal,0.5808,0.542424,"[0.674937965260546, 0.4099099099099099]",0.208568,0.722420,0.716581,"[0.7572614107883817, 0.6759002770083102]","[32, 32]",0.2,0.0001,1234,2,False,768


In [13]:
best_ffn_mean_history["f1"].mean()

0.5510875778045553

In [14]:
np.stack(best_ffn_mean_history["f1_scores"]).mean(axis=0)

array([0.69391722, 0.40825793])